In [1]:
"""
0. Connect to database
1. Navigate to SQL file + GUI
2. Read sql file
3. Query Data & write
    3.1 add datestamp
    3.2 finish file
"""

'\n0. Connect to database\n1. Navigate to SQL file + GUI\n2. Read sql file\n3. Query Data & write\n    3.1 add datestamp\n    3.2 finish file\n'

In [2]:
'''0. Connect to database'''

import pyodbc 
import pandas as pd
from datetime import datetime

# odbc connection info
driver = 'ODBC Driver 17 for SQL Server'
server = 'SQL-DSHE-PI-REPT'
db = 'USER_SUPPLY_CHAIN'

# establish connection
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+db+';Trusted_Connection=yes')

In [3]:
'''1. Navigate to SQL file + GUI'''

import PySimpleGUI as sg
sg.theme("DarkTeal2")
layout = [[sg.T("")], [sg.Text("Choose a file: "), sg.Input(), sg.FileBrowse(key="-IN-",file_types=(("SQL Files", "*.sql"),) )],[sg.Button("Submit")]]

###Building Window
window = sg.Window('My File Browser', layout, size=(600,150))
    
# get sql file
filePath = ''
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event=="Exit":
        break
    elif event == "Submit":
        filePath = values["-IN-"]
        window.close()

In [8]:
'''2. Read sql file'''

with open(filePath) as f:
    sql_as_string = f.read()

# remove potential encoding
val = sql_as_string.rfind('¿')
val = val+1
query=sql_as_string[val:]

In [10]:
'''3. Query Data & write'''

# query data
df = pd.read_sql_query(query,cnxn)

# save data with run date stamp
now = datetime.now()
currentTime = now.strftime("%m-%d-%y")
currentTime = currentTime+' at '+now.strftime("%H%M")
print(currentTime)

# set path to where sql file was
writeLoc = filePath[:len(filePath)-4] +'_'+ format(currentTime) +'.xlsx'
# set path to downloads
###writeLoc = filePath[:len(filePath)-4] +'_'+ format(currentTime) +'.xlsx'
print(writeLoc)

# save file to history
df.to_excel(writeLoc)

'''3.1 add datestamp'''
# add tab to new excel file with date
from openpyxl import load_workbook
wb2 = load_workbook(writeLoc)
wb2.create_sheet('dateStamp')

# add date to excel fiel
wb2['dateStamp'].cell(row = 1, column = 1).value = 'Date'
wb2['dateStamp'].cell(row = 2, column = 1).value = now

# save file
wb2.save(writeLoc)
wb2.close()

'''3.2 finish'''
# print runtime duration
runTime = (datetime.now()-now).seconds
print(runTime, 'seconds (about' , int(runTime/60), 'minutes)')

# COMPLETE
cnxn.close()
print('COMPLETED- Saved')
input('Press Enter or Close this window')

TypeError: 'NoneType' object is not iterable